In [968]:
import os
import numpy as np
import pandas as pd
import scipy.stats as stats
import warnings
import math
from scipy import stats
from collections import Counter

warnings.filterwarnings("ignore")

### weight of evidence ------

from https://stackoverflow.com/questions/50352210/woe-and-iv-table-in-python

In [969]:
def calc_iv(df, feature, target, pr=0):
    lst = []

    for i in range(df[feature].nunique()):
        val = list(df[feature].unique())[i]
        lst.append([feature, val, df[df[feature] == val].count()[feature], df[(df[feature] == val) & (df[target] == 1)].count()[feature]])
 
    data = pd.DataFrame(lst, columns=['Variable', 'Value', 'Pop', 'Bad'])
    data['Good'] = data['Pop'] - data['Bad']
    
    data['Badx'] = data['Bad'] + 0.01
    data['Goodx'] = data['Good'] + 0.01
    
    data['Pop'] = data['Good'] + data['Bad']
    data['Popx'] = data['Goodx'] + data['Badx']
    
    data['Dist Good'] = round((data['Pop'] - data['Bad']) / (data['Pop'].sum() - data['Bad'].sum()), 4)
    data['Dist Bad'] = round(data['Bad'] / data['Bad'].sum(), 4)
    
    data['Dist Goodx'] = (data['Popx'] - data['Badx']) / (data['Popx'].sum() - data['Badx'].sum())
    data['Dist Badx'] = data['Badx'] / data['Badx'].sum()
    
    data['Dist Pop'] = round(data['Pop'] / data['Pop'].sum(), 4)
    data['Bad Rate'] = round(data['Bad'] / data['Pop'], 4)
    data['grp_score'] = round((data['Dist Good']/(data['Dist Good'] + data['Dist Bad']))*10, 2)
    
    data['WoE'] = np.log(data['Dist Goodx'] / data['Dist Badx'])
    data['IV'] = data['WoE'] * (data['Dist Goodx'] - data['Dist Badx'])
    
    data['Efficiency'] =  abs(data['Dist Good'] - data['Dist Bad'])/2  
    
    data = data.sort_values(by=['Variable', 'Value'], ascending=True)
    data = data.reset_index()
    mydf = data[[
       'Value',
       'Good', 'Bad',
       'Dist Good','Dist Bad',
       'WoE',
       'IV'
    ]]

    #mydf=pd.DataFrame(data=d)
   
    if pr == 1:
        print(mydf)
        
    total_iv = data['IV'].sum()
    print(total_iv)

    #return data['IV'].values[0]
    #return mydf.values
   

In [970]:
iris = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', header=None)
iris.columns = ['sepal_l', 'sepal_w', 'petal_l', 'petal_w', 'type']

df = pd.get_dummies(iris.type)['Iris-setosa'].to_frame()
iris = iris.merge(df, left_index=True, right_index=True)

iris = iris.drop(['type','sepal_w','petal_l','petal_w'], axis=1)

In [971]:
iris = iris.rename(columns = {'Iris-setosa': 'target'})

In [972]:
print(iris.head())

   sepal_l  target
0      5.1       1
1      4.9       1
2      4.7       1
3      4.6       1
4      5.0       1


In [1045]:
t = pd.DataFrame(np.random.normal(1, 1, 1000))
t['target'] = 0
u = pd.DataFrame(np.random.normal(2, 1, 1000))
u['target'] = 1
data = pd.concat([t, u])
data.columns = ['x', 'target']
data.head()

,x,target
0,0.593408,0
1,-0.476526,0
2,1.397855,0
3,2.068536,0
4,1.735488,0


In [1046]:
data['cat'] = pd.qcut(data['x'], 100)
data['tran'] = data['cat'].apply(lambda x : x.left) 
data.head()

,x,target,cat,tran
0,0.593408,0,"(0.573, 0.608]",0.573
1,-0.476526,0,"(-0.542, -0.391]",-0.542
2,1.397855,0,"(1.387, 1.407]",1.387
3,2.068536,0,"(2.052, 2.08]",2.052
4,1.735488,0,"(1.719, 1.74]",1.719


In [1047]:
WOE_Binning(data, 'target', 'tran', p_threshold=0.2)

                Value  Good  Bad  Dist Good  Dist Bad       WoE        IV
0      [-inf, -0.542)    71    9      0.071     0.009  2.064486  0.127984
1   [-0.542, -0.0583)    84   16      0.084     0.016  1.657722  0.112713
2    [-0.0583, 0.761)   256   84      0.256     0.084  1.114281  0.191635
3      [0.761, 1.055)    97   63      0.097     0.063  0.431521  0.014670
4      [1.055, 1.185)    54   26      0.054     0.026  0.730688  0.020457
5      [1.185, 1.519)   120  100      0.120     0.100  0.182305  0.003646
6      [1.519, 1.626)    33   27      0.033     0.027  0.200603  0.001203
7       [1.626, 1.74)    36   44      0.036     0.044 -0.200620  0.001605
8       [1.74, 2.052)   106  134      0.106     0.134 -0.234381  0.006562
9      [2.052, 3.052)   115  345      0.115     0.345 -1.098554  0.252640
10       [3.052, inf)    28  152      0.028     0.152 -1.691385  0.209709
0.9428233065765501


based loosely on https://github.com/jstephenj14/Monotonic-WOE-Binning-Algorithm/blob/master/Monotonic%20WOE%20Binning.py

In [1028]:
def WOE_Binning(data, Y, attr, sign = True, p_threshold=0.05):
    defaults_threshold = 1
    #column = dataset.columns[dataset.columns != Y][0]
    column = data[attr]
    summary = data.groupby([column]).agg({Y:{"means":"mean","nsamples":"size","std_dev":"std"}})
    summary.columns = summary.columns.droplevel(level=0)
    summary = summary[["means","nsamples","std_dev"]]
    summary = summary.reset_index()
    summary["del_flag"] = 0
    summary["std_dev"] = summary["std_dev"].fillna(0)
    summary = summary.sort_values([attr],ascending = sign)
    summary.reset_index(inplace = True, drop = True) 
    
    i = 0
    while True:
        if i > len(summary)-2:
            break
        else:
            i = i + 1;
            x = i - 1;
           
            n_x = summary.iloc[x].nsamples
            n_i = summary.iloc[i].nsamples
            
            n = n_x + n_i
            mean_x = summary.iloc[x].means
            mean_i = summary.iloc[i].means
            
            var_x = (summary.iloc[x].std_dev)**2
            var_i = (summary.iloc[i].std_dev)**2
            
            #std deviation
            s = np.sqrt((var_x/n_x + var_i/n_i)/2)
            p = 0
            if s != 0:
                t = (mean_x - mean_i)/s
                df = 2*(n_x + n_i) - 2
                p = 1 - stats.t.cdf(t,df=df)
            if (p >= p_threshold) or (s == 0):
                #absorb data
                summary.loc[i, attr] = summary.loc[x, attr]
                summary.loc[i, 'nsamples'] = n_x + n_i
                summary.loc[i, 'means'] = (n_x*mean_x + n_i*mean_i)/n
                summary.loc[i, 'std_dev'] = np.sqrt(((n_x*var_x)+(n_i*var_i))/n)
                summary.loc[x, 'del_flag'] = 1        
            continue
            
    summary = summary[summary['del_flag'] == 0]
    summary = summary.reset_index()
    l = [-math.inf]
    for index, row in summary.iterrows():
        if index > 0:
            l.append(row[attr])
    l.append(math.inf)    
    #print(l)
    data['mybin'] = pd.cut(x=data[attr], bins=l, right=False)
    calc_iv(data, 'mybin', 'target', pr=1)
 

In [977]:
WOE_Binning(iris, 'target', 'sepal_l', p_threshold=0.2)

         Value  Good  Bad  Dist Good  Dist Bad       WoE        IV
0  [-inf, 4.9)     0   16       0.00      0.32 -8.071031  2.580956
1   [4.9, 5.5)     7   29       0.07      0.58 -2.112950  1.076665
2   [5.5, 5.6)     5    2       0.05      0.04  0.220654  0.002188
3   [5.6, 5.9)    18    3       0.18      0.06  1.096340  0.131418
4   [5.9, inf)    70    0       0.70      0.00  8.161161  5.709143
9.500369894639753


got this from github...

In [975]:
def chimerge(data, attr, label, max_intervals):
    distinct_vals = sorted(set(data[attr])) # Sort the distinct values
    labels = sorted(set(data[label]))       # Get all possible labels
    empty_count = {l: 0 for l in labels}    # A helper function for padding the Counter()
    intervals = [[distinct_vals[i], distinct_vals[i]] for i in range(len(distinct_vals))] # Initialize the intervals for each attribute
    while len(intervals) > max_intervals:   # While loop  
        chi = []
        for i in range(len(intervals)-1):
            # Calculate the Chi2 value
            obs0 = data[data[attr].between(intervals[i][0], intervals[i][1])]
            obs1 = data[data[attr].between(intervals[i+1][0], intervals[i+1][1])]
            total = len(obs0) + len(obs1)
            count_0 = np.array([v for i, v in {**empty_count, **Counter(obs0[label])}.items()])
            count_1 = np.array([v for i, v in {**empty_count, **Counter(obs1[label])}.items()])
            count_total = count_0 + count_1
            expected_0 = count_total*sum(count_0)/total
            expected_1 = count_total*sum(count_1)/total
            chi_ = (count_0 - expected_0)**2/expected_0 + (count_1 - expected_1)**2/expected_1
            chi_ = np.nan_to_num(chi_) # Deal with the zero counts
            chi.append(sum(chi_))      # Finally do the summation for Chi2
        min_chi = min(chi)             # Find the minimal Chi2 for current iteration
        for i, v in enumerate(chi): 
            if v == min_chi:
                min_chi_index = i      # Find the index of the interval to be merged
                break
        new_intervals = []             # Prepare for the merged new data array
        skip = False
        done = False
        for i in range(len(intervals)):
            if skip:
                skip = False
                continue
            if i == min_chi_index and not done: # Merge the intervals
                t = intervals[i] + intervals[i+1]
                new_intervals.append([min(t), max(t)])
                skip = True
                done = True
            else:
                new_intervals.append(intervals[i])
        intervals = new_intervals
        
   
    l = [-math.inf]
    for i in intervals:
        l.append(i[1])
    l.append(math.inf)
    print(l)   
    data['mybin'] = pd.cut(x=data[attr], bins=l, right=False)
    calc_iv(data, 'mybin', 'target', pr=1)
    #return l

In [976]:
chimerge(data=iris, attr='sepal_l', label='target', max_intervals=5)

[-inf, 4.8, 5.4, 5.5, 5.8, 7.9, inf]
         Value  Good  Bad  Dist Good  Dist Bad       WoE        IV
0  [-inf, 4.8)     0   11       0.00      0.22 -7.696522  1.691974
1   [4.8, 5.4)     6   29       0.06      0.58 -2.266764  1.177449
2   [5.4, 5.5)     1    5       0.01      0.10 -2.294033  0.206431
3   [5.5, 5.8)    17    4       0.17      0.08  0.752462  0.067642
4   [5.8, 7.9)    75    1       0.75      0.02  3.615123  2.637140
5   [7.9, inf)     1    0       0.01      0.00  3.922573  0.038811
5.819446171387975


testing ....

In [715]:
## Import the packages
import numpy as np
from scipy import stats

In [716]:
## Define 2 random distributions
#Sample Size
N = 10
#Gaussian distributed data with mean = 2 and var = 1
a = np.random.randn(N) + 2
#Gaussian distributed data with with mean = 0 and var = 1
b = np.random.randn(N)

In [726]:
## Calculate the Standard Deviation
#Calculate the variance to get the standard deviation

#For unbiased max likelihood estimate we have to divide the var by N-1, and therefore the parameter ddof = 1
var_a = a.var(ddof=1)
var_b = b.var(ddof=1)

In [728]:
print(var_a, var_b)

0.36602433831146336 0.7540508106223665


In [731]:
#std deviation
s = np.sqrt((var_a + var_b)/2)
#s

## Calculate the t-statistics
t = (a.mean() - b.mean())/(s*np.sqrt(2/N))

## Compare with the critical t-value
#Degrees of freedom
df = 2*N - 2

#p-value after comparison with the t 
p = 1 - stats.t.cdf(t,df=df)

print("t = " + str(t))
print("p = " + str(2*p))
### You can see that after comparing the t statistic with the critical t value (computed internally) we get a good p value of 0.0005 and thus we reject the null hypothesis and thus it proves that the mean of the two distributions are different and statistically significant.

## Cross Checking with the internal scipy function
t2, p2 = stats.ttest_ind(a,b)
print("t = " + str(t2))
print("p = " + str(p2))

t = 4.20921246738193
p = 0.0005273169029311742
t = 4.20921246738193
p = 0.0005273169029312183
